In [1]:
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

ACCESS_ID = ''
SECRET_ACCESS_KEY = ''
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False

In [83]:
def draw(model,dataset):
    fig = plt.figure(figsize = [5,5])
    ax = fig.add_subplot(1,1,1)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel('code_contribution_score')
    plt.ylabel('participation_score')
    colors = ['red','green','blue','black']
    
    k=0
    for i in model.labels_:
        plt.scatter(dataset[k][0], dataset[k][1], c=colors[i], marker='.',label=i)
        k+=1
    #plt.savefig('result'+str(index)+'.jpg')
    plt.show()

def Clustering(data):
    k = 2
    x = data.values * 1
    dataset = x[:,1:3]
    model = KMeans(n_clusters=k).fit(dataset)
    draw(model,dataset)

In [46]:
# 设置时间尺度
# paddlepaddle 起始、结束时间
begin_at = datetime.datetime(2016,8,1,0,0,0)
end_at = datetime.datetime(2021,11,1,0,0,0)

delta = end_at-begin_at
total_date = delta.days
interval = 90 #时间间隔
times = int(total_date / interval)
index = 0

In [2]:
#创建临时表单  paddlepaddle
TEMP_TABLE_NAME = 'temp_paddlepaddle_actors_activities_count_2'
drop_sql = '''
    DROP TABLE IF EXISTS {TEMP_TABLE_NAME};
    CREATE TABLE IF NOT EXISTS {TEMP_TABLE_NAME}
    (
        actor_id    STRING,
        icccount    STRING,
        oiccount    STRING,
        opccount STRING,
        rcccount STRING,
        mpccount STRING
    );
'''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME
)
o.execute_sql(drop_sql, hints={'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

<Instance 20220518153518126g6i1pta2>

In [7]:
PT1 = '20210701'
PT2 = '20211001'
repo_name = 'PaddlePaddle/Paddle'

select_sql_1 = '''
    INSERT INTO {TEMP_TABLE_NAME}
    SELECT
        a.actor_id AS actor_id,
        max(icc.icccount),
        max(oic.oiccount),
        max(opc.opccount),
        max(rcc.rcccount),
        max(mpc.mpccount)
    FROM
    (
        SELECT  actor_id
        FROM    ods_github_log
        WHERE   pt>='{pt1}'
        AND pt<='{pt2}'
        AND repo_name ='{repo_name}'
        GROUP BY    actor_id
    ) AS a
    FULL JOIN (
        SELECT
            actor_id,
            COUNT(*) AS icccount
        FROM ods_github_log
        WHERE (type = 'IssueCommentEvent') AND (action = 'created')
        AND repo_name='{repo_name}'
        AND pt>='{pt1}'
        AND pt<='{pt2}'
        GROUP BY
            actor_id
    ) AS icc ON (a.actor_id = icc.actor_id)
    FULL JOIN
    (
        SELECT
            actor_id,
            COUNT(*) AS oiccount
        FROM ods_github_log
        WHERE (type = 'IssuesEvent') AND (action = 'opened')
        AND repo_name='{repo_name}'
        AND pt>='{pt1}'
        AND pt<='{pt2}'
        GROUP BY
            actor_id
    ) AS oic ON (a.actor_id = oic.actor_id)
    FULL JOIN
    (
        SELECT
            actor_id,
            COUNT(*) AS opccount
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened')
        AND repo_name='{repo_name}'
        AND pt>='{pt1}'
        AND pt<='{pt2}'
        GROUP BY
            actor_id
    ) AS opc ON (a.actor_id = opc.actor_id)
    FULL JOIN
    (
        SELECT
            actor_id,
            COUNT(*) AS rcccount
        FROM ods_github_log
        WHERE (type = 'PullRequestReviewCommentEvent') AND (action = 'created')
        AND repo_name='{repo_name}'
        AND pt>='{pt1}'
        AND pt<='{pt2}'
        GROUP BY
            actor_id
    ) AS rcc ON (a.actor_id = rcc.actor_id)
    FULL JOIN
    (
        SELECT
            issue_author_id AS actor_id,
            COUNT(*) AS mpccount
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'closed') AND (pull_merged = 1)
        AND repo_name ='{repo_name}'
        AND pt>='{pt1}'
        AND pt<='{pt2}'
        GROUP BY
            issue_author_id
    ) AS mpc ON (a.actor_id = mpc.actor_id)
    GROUP BY
    a.actor_id;
    '''.format(
        pt1=PT1,
        pt2=PT2,
        repo_name=repo_name,
        TEMP_TABLE_NAME=TEMP_TABLE_NAME
    )

table = o.execute_sql(select_sql_1)


In [ ]:
# 按时间尺度查询数据 聚类
PT = '20160801'
repo_name = 'PaddlePaddle/Paddle'
for i in range(times):
    if(i == 0):
        begin_at_str = begin_at.strftime('%Y-%m-%d %H:%M:%S')
    add_date = (i + 1) * interval
    delta = datetime.timedelta(days=add_date)
    n_days = begin_at + delta
    end_at_str = n_days.strftime('%Y-%m-%d %H:%M:%S')
    # 此次起始、结束时间
    print(begin_at_str)
    print(end_at_str)
    # sql语句

    select_sql_1 = '''
    INSERT INTO {TEMP_TABLE_NAME}
    SELECT
        a.actor_id AS actor_id,
        icc.icccount,
        oic.oiccount,
        opc.opccount,
        rcc.rcccount,
        mpc.mpccount,   
        '{end_at_str}' AS end_date
    FROM
    (
        SELECT  actor_id
        FROM    ods_github_log
        WHERE   pt>'{pt1}'
        AND PT<'{pt2}'
        AND repo_name ='{repo_name}'
        GROUP BY    actor_id
    ) AS a
    FULL JOIN (
        SELECT
            actor_id,
            COUNT(*) AS icccount
        FROM ods_github_log
        WHERE (type = 'IssueCommentEvent') AND (action = 'created')
        AND repo_name='{repo_name}'
        AND pt>'{pt1}'
        AND PT<'{pt2}'
        AND created_at >= '{begin_at_str}'
        AND created_at <= '{end_at_str}'
        GROUP BY
            actor_id
    ) AS icc ON (a.actor_id = icc.actor_id)
    FULL JOIN
    (
        SELECT
            actor_id,
            COUNT(*) AS oiccount
        FROM ods_github_log
        WHERE (type = 'IssuesEvent') AND (action = 'opened')
        AND repo_name='{repo_name}'
        AND pt>'{pt1}'
        AND PT<'{pt2}'
        AND created_at >= '{begin_at_str}'
        AND created_at <= '{end_at_str}'
        GROUP BY
            actor_id
    ) AS oic ON (a.actor_id = oic.actor_id)
    FULL JOIN
    (
        SELECT
            actor_id,
            COUNT(*) AS opccount
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'opened')
        AND repo_name='{repo_name}'
        AND pt>'{pt1}'
        AND PT<'{pt2}'
        AND created_at >= '{begin_at_str}'
        AND created_at <= '{end_at_str}'
        GROUP BY
            actor_id
    ) AS opc ON (a.actor_id = opc.actor_id)
    FULL JOIN
    (
        SELECT
            actor_id,
            COUNT(*) AS rcccount
        FROM ods_github_log
        WHERE (type = 'PullRequestReviewCommentEvent') AND (action = 'created')
        AND repo_name='{repo_name}'
        AND pt>'{pt1}'
        AND PT<'{pt2}'
        AND created_at >= '{begin_at_str}'
        AND created_at <= '{end_at_str}'
        GROUP BY
            actor_id
    ) AS rcc ON (a.actor_id = rcc.actor_id)
    FULL JOIN
    (
        SELECT
            issue_author_id AS actor_id,
            COUNT(*) AS mpccount
        FROM ods_github_log
        WHERE (type = 'PullRequestEvent') AND (action = 'closed') AND (pull_merged = 1)
        AND repo_name ='{repo_name}'
        AND pt>'{pt1}'
        AND PT<'{pt2}'
        AND created_at >= '{begin_at_str}'
        AND created_at <= '{end_at_str}'
        GROUP BY
            issue_author_id
    ) AS mpc ON (a.actor_id = mpc.actor_id)
    GROUP BY
    a.actor_id;
    '''.format(
        pt=PT,
        begin_at_str=begin_at_str,
        end_at_str=end_at_str,
        repo_name=repo_name,
    )

    table = o.execute_sql(select_sql_1)
    begin_at_str = end_at_str


In [59]:
select_sql_2 = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT  actor_id
        ,icccount
        ,oiccount
        ,opccount
        ,rcccount
        ,mpccount
        ,wccount
        ,fccount
        ,code_contribution_score
        ,participation_score
        ,end_date
    FROM    {TEMP_TABLE_NAME}
    WHERE     code_contribution_score >0
    AND     participation_score > 0
    ORDER BY    end_date
    '''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME
)

In [12]:
select_sql_3 = '''
    set odps.sql.validate.orderby.limit=false;
    SELECT  actor_id
        ,icccount
        ,oiccount
        ,opccount
        ,rcccount
        ,mpccount
    FROM    {TEMP_TABLE_NAME}
    '''.format(
    TEMP_TABLE_NAME=TEMP_TABLE_NAME
)

In [14]:
table = o.execute_sql(select_sql_3, hints={
                        'odps.sql.allow.fullscan': 'true', 'odps.sql.submit.mode': 'script'})

data = pd.DataFrame(
    columns=['actor_id', 'icccount', 'oiccount', 'opccount', 'rcccount', 'mpccount'])
i = 0
with table.open_reader() as reader:
    for record in reader:
        data.loc[i] = [record[0], record[1], record[2], record[3],record[4], record[5]]
        i += 1
#

In [19]:
data.to_csv('paddlepaddle_actor_acticities_202110.csv')

In [63]:
data.sort_values(by='end_date', inplace=True, ascending=True)
data

,actor_id,icccount,oiccount,opccount,rcccount,mpccount,code_contribution_score,participation_score,end_date
0,22388454,14,3,1,None,1,20.0,8.0,2017-01-28 00:00:00
24,18067400,3,1,2,None,2,5.0,16.0,2017-01-28 00:00:00
25,21339431,64,13,14,93,14,276.0,112.0,2017-01-28 00:00:00
26,1783632,4,3,4,1,1,12.0,17.0,2017-01-28 00:00:00
27,2063642,2,None,3,None,1,2.0,14.0,2017-01-28 00:00:00
...,...,...,...,...,...,...,...,...,...
1636,24504497,None,None,11,111,7,222.0,68.0,2021-10-04 00:00:00
1637,24829556,None,None,24,12,13,24.0,137.0,2021-10-04 00:00:00
1638,25279174,8,None,35,52,23,112.0,220.0,2021-10-04 00:00:00
1630,23398086,None,None,4,1,3,2.0,27.0,2021-10-04 00:00:00


In [89]:
dataset = data.loc[data['end_date'] == '2021-10-04 00:00:00']

In [ ]:
# 设置权重
# 代码贡献
MERGEPULL_WEIGHT = 5 #mpc
OPENPULL_WEIGHT = 4 #opc
PULLREVIEWCOMMENT_WEIGHT = 2 #rcc
# 参与&讨论
OPENISSUE_WEIGHT = 3 #oic
ISSUECOMMENT_WEIGHT = 2 #icc
WATCH_WEIGHT = 1 #wc
FORK_WEIGHT = 1 #fc